In [5]:
import pandas as pd
from datetime import datetime, timedelta

In [6]:
# Cargar el dataset desde un archivo .xlsx
df = pd.read_excel('Datos_15-11-22_al_30-05-23.xlsx')

In [3]:
# Asegurarnos de que la columna 'Fecha' esté en formato de fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%m/%d/%Y')

# Ordenar el DataFrame por fecha
df = df.sort_values('Fecha').reset_index(drop=True)

# Definir los parámetros del cultivo (trigo)
pcc = 140  # Periodo de crecimiento del cultivo en días
kc_values = {
    'inicio': 0.15,  # Kc inicial
    'medio': 1.15,   # Kc medio
    'final': 0.50    # Kc final
}

# Definir la fecha de inicio en el código
fecha_inicio_str = "11/30/2022"  # Define aquí la fecha en formato mm/dd/aaaa
fecha_inicio = datetime.strptime(fecha_inicio_str, '%m/%d/%Y')

# Verificar si hay suficientes días para el periodo de cultivo
fecha_fin = fecha_inicio + timedelta(days=pcc - 1)
ultima_fecha = df['Fecha'].max()

if fecha_fin > ultima_fecha:
    print(f"No hay suficientes datos para un periodo de {pcc} días desde la fecha de inicio {fecha_inicio_str}.")
else:
    # Filtrar el DataFrame para el periodo de 140 días desde la fecha de inicio y hacer una copia para evitar warnings
    df_periodo = df[(df['Fecha'] >= fecha_inicio) & (df['Fecha'] <= fecha_fin)].copy()

    # Crear y asignar la columna 'Kc' como float para evitar problemas de tipo de datos
    df_periodo['Kc'] = 0.0
    df_periodo.loc[df_periodo.index < 45, 'Kc'] = kc_values['inicio']
    df_periodo.loc[(df_periodo.index >= 45) & (df_periodo.index < 95), 'Kc'] = kc_values['medio']
    df_periodo.loc[df_periodo.index >= 95, 'Kc'] = kc_values['final']

    # Calcular ETc (Evapotranspiración de cultivo)
    df_periodo['ETc'] = df_periodo['Kc'] * df_periodo['ETO']

    # Definir la función para calcular Precipitación efectiva
    def calcular_precip_efectiva(precipitacion_total):
        if precipitacion_total < 250:
            # Cálculo de la precipitación efectiva cuando es menor a 250 mm
            precip_efectiva = precipitacion_total * (125 - 0.2 * precipitacion_total) / 125
        else:
            # Cálculo de la precipitación efectiva cuando es mayor o igual a 250 mm
            precip_efectiva = 125 + 0.1 * precipitacion_total
        return precip_efectiva

    # Calcular Precipitación efectiva para cada registro
    df_periodo['Precipitación efectiva'] = df_periodo['Precipitación (mm)'].apply(calcular_precip_efectiva)

    # Calcular ETazul (Evapotranspiración de agua azul)
    df_periodo['ETazul'] = (df_periodo['ETc'] - df_periodo['Precipitación efectiva']).apply(lambda x: max(0, x))

    # Calcular el uso de agua azul total en el periodo de 140 días
    uso_agua_azul = 10 * df_periodo['ETazul'].sum()

    # Mostrar el resultado
    print(f"Uso de Agua Azul en Cultivo para los {pcc} días desde {fecha_inicio_str}: {uso_agua_azul} metros cubicos de agua por tonelada.")

Uso de Agua Azul en Cultivo para los 140 días desde 11/30/2022: 3360.9673999999995 metros cubicos por tonelada.


In [8]:
# Cargar el dataset desde un archivo .xlsx
df = pd.read_excel('Datos_15-11-22_al_30-05-23.xlsx')

# Asegurarnos de que la columna 'Fecha' esté en formato de fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%m/%d/%Y')

# Ordenar el DataFrame por fecha
df = df.sort_values('Fecha').reset_index(drop=True)

# Definir los parámetros del cultivo (trigo)
pcc = 140  # Periodo de crecimiento del cultivo en días
kc_values = {
    'inicio': 0.15,  # Kc inicial
    'medio': 1.15,   # Kc medio
    'final': 0.50    # Kc final
}

# Definir la fecha de inicio en el código
fecha_inicio_str = "11/30/2022"  # Define aquí la fecha en formato mm/dd/aaaa
fecha_inicio = datetime.strptime(fecha_inicio_str, '%m/%d/%Y')

# Verificar si hay suficientes días para el periodo de cultivo
fecha_fin = fecha_inicio + timedelta(days=pcc - 1)
ultima_fecha = df['Fecha'].max()

if fecha_fin > ultima_fecha:
    print(f"No hay suficientes datos para un periodo de {pcc} días desde la fecha de inicio {fecha_inicio_str}.")
else:
    # Filtrar el DataFrame para el periodo de 140 días desde la fecha de inicio y hacer una copia para evitar warnings
    df_periodo = df[(df['Fecha'] >= fecha_inicio) & (df['Fecha'] <= fecha_fin)].copy()

    # Crear y asignar la columna 'Kc' como float para evitar problemas de tipo de datos
    df_periodo['Kc'] = 0.0
    df_periodo.loc[df_periodo.index < 45, 'Kc'] = kc_values['inicio']
    df_periodo.loc[(df_periodo.index >= 45) & (df_periodo.index < 95), 'Kc'] = kc_values['medio']
    df_periodo.loc[df_periodo.index >= 95, 'Kc'] = kc_values['final']

    # Calcular ETc (Evapotranspiración de cultivo)
    df_periodo['ETc'] = df_periodo['Kc'] * df_periodo['ETO']

    # Definir la función para calcular Precipitación efectiva
    def calcular_precip_efectiva(precipitacion_total):
        if precipitacion_total < 250:
            # Cálculo de la precipitación efectiva cuando es menor a 250 mm
            precip_efectiva = precipitacion_total * (125 - 0.2 * precipitacion_total) / 125
        else:
            # Cálculo de la precipitación efectiva cuando es mayor o igual a 250 mm
            precip_efectiva = 125 + 0.1 * precipitacion_total
        return precip_efectiva

    # Calcular Precipitación efectiva para cada registro
    df_periodo['Precipitación efectiva'] = df_periodo['Precipitación (mm)'].apply(calcular_precip_efectiva)

    # Calcular ETverde (Evapotranspiración de agua verde)
    df_periodo['ETverde'] = df_periodo[['ETc', 'Precipitación efectiva']].min(axis=1)

    # Calcular el uso de agua verde total en el periodo de 140 días
    uso_agua_verde = 10 * df_periodo['ETverde'].sum()

    # Mostrar el resultado
    print(f"Uso de Agua Verde en Cultivo para los {pcc} días desde {fecha_inicio_str}: {uso_agua_verde} m³/ha")

Uso de Agua Verde en Cultivo para los 140 días desde 11/30/2022: 56.3126 m³/ha
